# 凯撒与哑谜（前传）

凯撒密码和哑谜都是文本加密的方法。这类方法还有很多。
为了更好的操作他们（以及破解），我们需要一套统一的接口：加密器类！

## 加密器类 `Cipher`（父类）


In [1]:
class Cipher:
    def reset(self):
        pass  # default no-op

    def encrypt(self, text):
        raise NotImplementedError

    def decrypt(self, text):
        raise NotImplementedError

## 凯撒加密器

In [3]:
class CaesarCipher(Cipher):
    # your code here
    def __init__(self, offset):
        self.offset = offset
    
    def reset(self):
        self.offset = 0
    
    def encrypt(self, text):
        new_text = ''
        for c in text:
            if c.isalpha():
                if c.isupper():
                    new_text += chr((ord(c) - 65 + self.offset) % 26 + 65)
                else:
                    new_text += chr((ord(c) - 97 + self.offset) % 26 + 97)
            else:
                new_text += c

        return new_text
        
    
    def decrypt(self, text):
        new_text = ''
        for c in text:
            if c.isalpha():
                if c.isupper():
                    new_text += chr((ord(c) - 65 - self.offset) % 26 + 65)
                else:
                    new_text += chr((ord(c) - 97 - self.offset) % 26 + 97)
            else:
                new_text += c
                
        return new_text
    
    pass

In [5]:
caesar_cipher = CaesarCipher(offset=3)
print(caesar_cipher.encrypt("Python"))
print(caesar_cipher.decrypt("Sbwkrq"))

# 预期结果：
# Sbwkrq
# Python

Sbwkrq
Python


## 哑谜加密器

### EnigmaMachine的使用


在Python中，有一些第三方库可以帮助你更容易地模拟恩尼格玛机的行为，而无需从头开始实现所有的逻辑。一个这样的库叫做 `enigma`，它提供了一个更为完整的恩尼格玛机模拟。

要使用 `enigma` 库中的 `EnigmaMachine` 类，你需要先安装该库。你可以通过pip来安装：


In [7]:
!pip install py-enigma

接下来，你可以按照以下步骤来使用 `EnigmaMachine`：

1. **导入所需的模块**：从 `enigma` 库中导入 `EnigmaMachine`
2. **创建机器实例**：根据需要的配置创建一个 `EnigmaMachine` 的实例。
3. **设置参数**：添加转子和反射器和插线板到机器实例。
4. **加密或解密消息**：使用 `process_text` 方法来加密或解密文本。

下面是一个具体的使用示例：

In [9]:
from enigma.machine import EnigmaMachine

machine = EnigmaMachine.from_key_sheet(
            rotors=["I","IV","II"],
            reflector="B",
            plugboard_settings="AB EZ CX DP KN TY",
        )
machine.set_display("aAc")
print(machine.process_text('Python'))
machine.set_display("aAc")
print(machine.process_text('VIXQTI'))

GVMKNP
NHKXHE


### 实现哑谜加密器

In [11]:
from enigma.machine import EnigmaMachine


class EnigmaCipher(Cipher):
    # your code here
    def __init__(self, rotors, reflector, plugboard, display):
        self.machine = EnigmaMachine.from_key_sheet(
            rotors=rotors,
            reflector=reflector,
            plugboard_settings=plugboard,
        )
        self.initial_display = display
        self.machine.set_display(display)

    
    def reset(self):
        self.machine.set_display(self.initial_display)
    
    def encrypt(self, text):
        return self.machine.process_text(text)
    
    def decrypt(self, text):
        return self.machine.process_text(text)
    pass

In [13]:
enigma_cipher = EnigmaCipher(
    rotors=["I","IV","II"],
    reflector="B",
    plugboard="AB EZ CX DP KN TY",
    display="ZEN")
print(enigma_cipher.encrypt("Python"))
print(enigma_cipher.decrypt("VIXQTI"))
enigma_cipher.reset()
print(enigma_cipher.decrypt("VIXQTI"))

# 预期结果：
# VIXQTI
# GUFUVB
# PYTHON

VIXQTI
GUFUVB
PYTHON
